### 1. 데이터 영어명 한글로 바꾸기.
- #### 1) 주 이름 (**customer** 데이터)
- #### 2) 상품명 (**product_name** 데이터)
### 2. 지리정보 위도/경도 (**customer**  데이터) --> 대표 위도/경도로 표시
### 3. 매출/이익 각각 계산 (**item** 데이터)--> Sales와 Profit 계산을 위해서 어떤 컬럼을 사용하고 계산해야하는지 확인
- #### 1) profit 컬럼 새로 만듦 (item_count * price)
- #### 2) 중복되는 행 삭제(최대구매갯수 행만 남김)
### 4. datetime 유형으로 배달시간 데이터 타입변경 (**orders** 데이터)
- #### 1) 주문, 배달 시간 관련된 컬럼들의 유형이 object이므로 datetime유형으로 변환 후 시간비교(오류검수)
### 5. **review** 데이터에서 필요한 컬럼만 남기고 나머지는 drop

In [2]:
import pandas as pd

## 3. 매출/이익 계산을 위해 payment_value가 order_item의 계산 합과 일치하는지 확인.

- order_id, 00143d0f86d6fbd9f9b38ab440ac16f5 는 3개의 아이템을 구매.
  - The total order_item value is: 21.33 * 3 = 63.99
  - The total freight value is: 15.10 * 3 = 45.30
  - The total order value (product + freight) is: 45.30 + 63.99 = 109.29

In [24]:
order_items = pd.read_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/olist_order_items.csv')

In [27]:
order_items[order_items['order_id'] == '00143d0f86d6fbd9f9b38ab440ac16f5']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
32,00143d0f86d6fbd9f9b38ab440ac16f5,1,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.1
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.1
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.1


In [17]:
payments = pd.read_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/olist_order_payments.csv')

In [61]:
payments.head(1)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33


In [30]:
payments[payments['order_id'] == '00143d0f86d6fbd9f9b38ab440ac16f5']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
29413,00143d0f86d6fbd9f9b38ab440ac16f5,1,credit_card,1,109.29


- Sales 계산시 payment_value 사용

In [62]:
payments.to_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/cleaned_csv/payments.csv')

- Profit 계산하기 위해 order_item 데이터셋에서 최대구매갯수 행만 남기고 중복되는 행 제거
  - 새로운 테이블명: items

In [42]:
result = order_items.loc[order_items.groupby('order_id')['order_item_id'].idxmax()]
items = result
items.shape

(98666, 7)

- items 테이블에서 profit 컬럼 새로 만듦

In [46]:
items['profit'] = items['order_item_id']*items['price']

- items 테이블에서 order_item_id 컬럼명을 알아보기 쉽게 아이템 개수 컬럼이름으로 변경 (item_count) 

In [53]:
items = items.rename(columns={'order_item_id': 'item_count'})

In [55]:
items.head(1)

,order_id,item_count,product_id,seller_id,shipping_limit_date,price,freight_value,profit
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,58.9


In [60]:
items.to_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/items.csv')

## 4. datetime 유형으로 배달시간 변경 & 오류 검수

In [33]:
orders = pd.read_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/olist_orders.csv')

In [35]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [38]:
# column 3, 4, 5, 6, 7 datetime으로 변환
orders[orders.columns[[3,4,5,6,7]]] = orders[orders.columns[[3,4,5,6,7]]].apply(pd.to_datetime)

print(orders.dtypes)

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object


In [40]:
orders.head(1)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18


- 주문 중 주문날짜 오류 검수
  1) order_purchase_timestamp > order_approved_at --> 오류없음
  2) order_approved_at > order_delivered_carrier_date --> 오류있음 (1359건)
     - 1359건 삭제 후 3번 검수 진행    
  3) order_delivered_carrier_date > order_delivered_customer_date

In [63]:
orders[orders["order_purchase_timestamp"] > orders["order_approved_at"]]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [65]:
error_date_1 = orders[orders["order_approved_at"] > orders["order_delivered_carrier_date"]]

In [66]:
error_date_1.shape

(1359, 8)

- 주문이 운송업체에게 전달된 날짜가 주문승인된 날짜보다 이른 경우 - 1359건
  - 오류이므로 삭제

In [70]:
error_excluded = orders.merge(error_date_1, on=orders.columns.tolist(), how='left', indicator=True)
error_excluded = error_excluded[error_excluded['_merge'] == 'left_only'].drop('_merge', axis=1)

In [71]:
error_excluded.shape

(98082, 8)

In [74]:
error_date_2 = error_excluded[error_excluded["order_delivered_carrier_date"] > error_excluded["order_delivered_customer_date"]]

In [75]:
error_date_2.shape

(23, 8)

- 주문이 고객에게 도착한 날짜가 주문이 운송업체에게 전달된 날짜보다 이른 경우 - 23건
  - 오류이므로 삭제

In [76]:
error_excluded_2 = error_excluded.merge(error_date_2, on=error_excluded.columns.tolist(), how='left', indicator=True)
error_excluded_2 = error_excluded_2[error_excluded_2['_merge'] == 'left_only'].drop('_merge', axis=1)

In [77]:
error_excluded_2.shape

(98059, 8)

In [80]:
orders = error_excluded_2
print(orders.shape)

(98059, 8)


In [81]:
orders.to_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/cleaned_csv/orders.csv')

### 1-2) 영어 상품명 한국어로 바꾸기

In [2]:
english_product_name = pd.read_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/product_category_name_translation.csv')

In [3]:
english_product_name.head(2)

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories


In [4]:
# 기존 상품의 영어명을 한글명으로 매핑 (A->X, B->Y, C->Z)
mapping = {'health_beauty': '건강/뷰티', 'computers_accessories': '컴퓨터/액세서리', 'auto':'자동차', 
           'bed_bath_table': '침구/욕실/테이블', 'furniture_decor': '가구/인테리어', 'sports_leisure':'스포츠/레저',
           'perfumery': '향수', 'housewares':'주방용품', 'telephony':'통신기기', 'watches_gifts':'시계/선물', 
           'food_drink':'음식/음료', 'baby':'유아용품', 'stationery':'문구', 
           'tablets_printing_image': '태블릿/인쇄/이미지', 'toys':'장난감', 'fixed_telephony':'유선 전화',
           'garden_tools':'정원 도구', 'fashion_bags_accessories': '패션 가방/액세서리', 
           'small_appliances':'소형 가전', 'consoles_games': '게임 콘솔/게임', 'audio':'오디오', 
           'fashion_shoes':'패션 신발', 'cool_stuff':'특별 상품', 'luggage_accessories':'여행 가방/액세서리',
           'air_conditioning':'에어컨', 'construction_tools_construction':'건설 도구/건설', 
           'kitchen_dining_laundry_garden_furniture': '주방/식당/세탁/정원 가구', 
           'costruction_tools_garden':'건설 도구/정원', 'fashion_male_clothing':'남성 의류', 
           'pet_shop':'애완동물 용품', 'office_furniture':'사무용 가구', 'market_place':'마켓플레이스',
           'electronics':'전자제품', 'home_appliances':'가전제품', 'party_supplies':'파티 용품', 
           'home_confort':'홈 컴포트', 'costruction_tools_tools':'건설 도구', 
           'agro_industry_and_commerce': '농업/산업/상업', 'furniture_mattress_and_upholstery':'가구/매트리스/침구',
           'books_technical':'기술 서적', 'home_construction':'주택 건설', 'musical_instruments': '악기',
           'furniture_living_room':'거실 가구', 'construction_tools_lights':'건설 도구/조명',
           'industry_commerce_and_business':'산업/상업/비즈니스', 'food':'식품', 'art':'예술', 
           'furniture_bedroom':'침실 가구', 'books_general_interest':'일반 관심 서적', 
           'construction_tools_safety':'건설 도구/안전', 'fashion_underwear_beach': '언더웨어/비치웨어',
           'fashion_sport':'스포츠 의류', 'signaling_and_security':'신호/보안', 'computers':'컴퓨터', 
           'christmas_supplies':'크리스마스 용품', 'fashio_female_clothing':'여성 의류', 
           'home_appliances_2':'가전제품 2', 'books_imported':'수입 서적', 'drinks':'음료', 'cine_photo':'영화/사진',
           'la_cuisine':'주방 용품', 'music': '음악', 'home_comfort_2':'홈 컴포트 2', 
           'small_appliances_home_oven_and_coffee': '소형 가전/가정용 오븐/커피', 
           'cds_dvds_musicals':'CD/DVD/뮤지컬', 'dvds_blu_ray':'DVD/블루레이', 'flowers':'꽃', 
           'arts_and_craftmanship':'공예/수공예', 'diapers_and_hygiene':'기저귀/위생용품',
           'fashion_childrens_clothes':'아동 의류', 'security_and_services':'보안/서비스'}
english_product_name['korean'] = english_product_name['product_category_name_english'].map(mapping)

In [57]:
english_product_name.head(2)

,product_category_name,product_category_name_english,korean
0,beleza_saude,health_beauty,건강/뷰티
1,informatica_acessorios,computers_accessories,컴퓨터/액세서리


In [58]:
english_product_name.to_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/product_name.csv', encoding="utf-8-sig")

## 2. 위도/경도 대표 위치로 바꾸기

In [8]:
customer = pd.read_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/olist_customers_dataset.csv')

In [9]:
customer.head(2)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP


In [10]:
mapping = {'AC':-9.9747, 'AL':-9.6667, 'AP':0.0330, 'AM':-3.4653, 'BA':-12.9714, 'CE':-3.7172, 'ES':-20.3155, 
           'GO':-16.6869, 'MA':-2.5387, 'MT':-15.5989, 'MS':-20.7722, 'MG':-19.9167, 'PA':-1.4558, 
           'PB':-7.1200, 'PR':-25.4284, 'PE':-8.0476, 'PI':-5.0892, 'RJ':-22.9068, 'RN':-5.7945, 'RS':-30.0331, 
           'RO':-8.7617, 'RR':2.8195, 'SC':-27.5969, 'SP':-23.5505, 'SE':-10.9472, 'TO':-10.1845, 'DF':-15.7975}
customer['latitude'] = customer['customer_state'].map(mapping)

In [11]:
mapping = {'AC':-67.8100, 'AL':-35.7167, 'AP':-51.0654, 'AM':-62.2159, 'BA':-38.5014, 'CE':-38.5433, 
           'ES':-40.3128, 'GO':-49.2648, 'MA':-44.2828, 'MT':-56.0949, 'MS':-54.7852, 'MG':-43.9333, 
           'PA':-48.4902, 'PB':-34.8828, 'PR':-49.2733, 'PE':-34.8770, 'PI':-42.8036, 'RJ':-43.1729,
           'RN':-35.2110, 'RS':-51.2300, 'RO':-63.9019, 'RR':-60.6714, 'SC':-48.5495, 'SP':-46.6333,
           'SE':-37.0731, 'TO':-48.3337, 'DF':-47.8919}
customer['longitude'] = customer['customer_state'].map(mapping)

In [12]:
customer.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,latitude,longitude
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,-23.5505,-46.6333
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,-23.5505,-46.6333
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,-23.5505,-46.6333


### 1-1) Customer State --> 한국어 명칭으로 바꾸기

In [13]:
mapping = {'AL':'알라고아스', 'AC': '아크레', 'AM':'아마조나스', 'AP':'아마파', 'BA':'바이아', 'CE':'세아라', 'DF':'연방 지구',
           'ES':'에스피리투산투', 'GO':'고이아스', 'MA':'마라냥', 'MG':'미나스제라이스', 'MS':'마투그로수 두 술',
           'MT':'마투그로수', 'PA':'파라', 'PB':'파라이바', 'PE':'페르남부쿠', 'PI':'피아우이', 'PR':'파라나',
           'RJ':'리우데자네이루', 'RN':'히우그란지 두 노르치', 'RO':'히우그란지 두 술', 'RR':'호라이마', 
           'RS':'히우그란지 두 술', 'SC':'산타카타리나', 'SE':'세르지피', 'SP':'상파울루', 'TO':'토칸칭스'}
customer['kor_state'] = customer['customer_state'].map(mapping)

In [14]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               99441 non-null  object 
 1   customer_unique_id        99441 non-null  object 
 2   customer_zip_code_prefix  99441 non-null  int64  
 3   customer_city             99441 non-null  object 
 4   customer_state            99441 non-null  object 
 5   latitude                  99441 non-null  float64
 6   longitude                 99441 non-null  float64
 7   kor_state                 99441 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 6.1+ MB


In [15]:
customer.to_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/cleaned_csv/customers.csv', encoding="utf-8-sig", index = False)

In [84]:
reviews = pd.read_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/olist_order_reviews.csv')

In [85]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [86]:
reviews = reviews.drop(reviews.columns[3:7], axis=1)
print(reviews.shape)

(99224, 3)


In [87]:
reviews.head(3)

,review_id,order_id,review_score
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5


In [88]:
reviews.to_csv('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/cleaned_csv/reviews.csv')

In [3]:
df = pd.read_excel('C:/Users/elley/Desktop/Python/project_4_portfolio/ecommerce/excel/sales_order_item.xlsx')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97913 entries, 0 to 97912
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   order_id                       97913 non-null  object 
 1   customer_id                    97913 non-null  object 
 2   order_status                   97913 non-null  object 
 3   order_purchase_timestamp       97913 non-null  object 
 4   order_approved_at              97899 non-null  object 
 5   order_delivered_carrier_date   96276 non-null  object 
 6   order_delivered_customer_date  95103 non-null  object 
 7   order_estimated_delivery_date  97913 non-null  object 
 8   customer_unique_id             97913 non-null  object 
 9   customer_state                 97913 non-null  object 
 10  latitude                       97913 non-null  float64
 11  longitude                      97913 non-null  float64
 12  kor_state                      97913 non-null 

In [89]:
# 특정 컬럼에 중복된 값이 있는지 확인
duplicates = reviews['order_id'].duplicated()

# 중복된 값만 필터링
duplicates_only = reviews[duplicates]

# 중복된 값의 개수
# duplicates_count = duplicates.sum()

# print(duplicates_only)
# print(f"중복된 값의 개수: {duplicates_count}")


In [90]:
duplicates_only

,review_id,order_id,review_score
1119,46abf3ea0b2710ad41390fdb79c32d84,5040757d4e06a4be96d3827b860b4e7c,5
3109,aa193e76d35950c4ae988237bb36ed2b,cf73e2cb1f4a9480ed70c154da3d954a,5
8108,40294ea5a778dc62080d6b3f55d361ce,e1bc1083cd7acd30d0576335373b907d,5
9064,32e2c7e889f7a185d462265398ee3631,c7cfea0c153e6382e32e84c2a9dd7d2e,5
9795,95a3135743556b117d888cc8c6e12e11,f9c78e6e58306dc81efbbada1ac11f24,3
...,...,...,...
98989,dfb3db02188d809d5cd199496b6da87e,c0db7d31ace61fc360a3eaa34dd3457c,5
99108,2c6c08892b83ba4c1be33037c2842294,42ae1967f68c90bb325783ac55d761ce,4
99164,2afe63a67dfd99b3038f568fb47ee761,c5334d330e36d2a810a7a13c72e135ee,5
99178,44d1e9165ec54b1d89d33594856af859,a7dbcf5043158d6fa72859eead2f3d10,4


In [91]:
r_duplicate = reviews['review_id'].duplicated()
duplicated = reviews[r_duplicate]


In [92]:
duplicated

,review_id,order_id,review_score
3317,3242cc306a9218d0377831e175d62fbf,9c5bfba7de6a4abbb6ba0baab78d1622,5
5719,308316408775d1600dad81bd3184556d,3fe4dbcdb046a475dbf25463c1ca78bd,5
7213,8ee90ac383cf825bb7f4756130d4e74a,75d5d3d16567a27eefc5752aeb063072,5
7805,d433c252647c51309432ca0b763f969b,191abde9c3bf77d1ce67d100d8db6d15,1
9013,03a6a25db577d0689440933055111897,3fde8b7313af6b37b84b5c7594d7add0,5
...,...,...,...
99108,2c6c08892b83ba4c1be33037c2842294,42ae1967f68c90bb325783ac55d761ce,4
99124,6ec93e77f444e0b1703740a69122e35d,e1fdc6e9d1ca132377e862593a7c0bd4,5
99164,2afe63a67dfd99b3038f568fb47ee761,c5334d330e36d2a810a7a13c72e135ee,5
99167,017808d29fd1f942d97e50184dfb4c13,b1461c8882153b5fe68307c46a506e39,5


In [93]:
order_items.head(2)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93


In [94]:
order_duplicate = order_items['order_id'].duplicated()
duplicated = order_items[order_duplicate]


In [95]:
duplicated

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.90,13.37
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
43,001ab0a7578dd66cd4b0a71f5b6e1e41,2,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63
44,001ab0a7578dd66cd4b0a71f5b6e1e41,3,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63
...,...,...,...,...,...,...,...
112617,ffecd5a79a0084f6a592288c67e3c298,3,50fd2b788dc166edd20512370dac54df,8b321bb669392f5163d04c59e235e066,2018-03-05 20:15:27,21.90,15.79
112635,fff8287bbae429a99bb7e8c21d151c41,2,bee2e070c39f3dd2f6883a17a5f0da45,4e922959ae960d389249c378d1c939f5,2018-03-27 12:29:22,180.00,48.14
112641,fffb9224b6fc7c43ebb0904318b10b5f,2,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,2017-11-03 02:55:58,55.00,34.19
112642,fffb9224b6fc7c43ebb0904318b10b5f,3,43423cdffde7fda63d0414ed38c11a73,b1fc4f64df5a0e8b6913ab38803c57a9,2017-11-03 02:55:58,55.00,34.19
